In [25]:
import sys
sys.path.append('/home/xilini/webnlg/submissions2020/corpus-reader-master/')

from benchmark_reader import Benchmark
from benchmark_reader import select_files
import json

# extract reference texts
b = Benchmark()
b.fill_benchmark([('/home/xilini/webnlg/submissions2020/testdata/rdf2text/en/',
                   'rdf-to-text-generation-test-data-with-refs-en.xml')])
b.b2json('../../texts/', 'data_REF.json')

In [38]:
with open('../inputs/sampled_ids.txt', 'r') as f2:
    sampled_ids = f2.read().splitlines()

In [28]:
with open('../../texts/data_REF.json', 'r') as f1:
    data_REF = json.load(f1)['entries']

In [40]:
sampled_data_REF = []
for item in data_REF:
    for key, value in item.items():
        if key in sampled_ids:
            sampled_data_REF.append(item)

In [51]:
lexicalisations = {}
import random
random.seed(0)

for entry in sampled_data_REF:
    lexs = []
    for k, v in entry.items():
        lex = v['lexicalisations']
        for i in lex:
            lexs.append(i['lex'])
    this_lex = random.sample(lexs, 1)
    lexicalisations[k] = this_lex[0]

In [55]:
with open('../inputs/texts_REF.json', 'w') as f3:
    json.dump(lexicalisations, f3)

In [60]:
# prepare triples

triplet_data = {}

for entry in sampled_data_REF:
    lexs = []
    rows = []
    for k, v in entry.items():
        triples = v['modifiedtripleset']       
        for t in triples:
            subj = t['subject'].replace('_', ' ')
            pred = t['property'].replace('_', ' ')
            obj = t['object'].replace('_', ' ')
            this_t = [subj, pred, obj]
            rows.append(this_t)
                
    triplet_data[k] = []
    triplet_data[k].append(rows)


In [61]:
html_triplet = '''

  <tr>
    <td class="tg-0lax">SUBJ</td>
    <td class="tg-0lax">PRED</td>
    <td class="tg-0lax">OBJ</td>
  </tr>

'''

In [62]:
html_head = '''

<div class="center">
<table class="tg">

<thead>
  <tr>
    <th class="tg-0pky"><b>Subject</b></th>
    <th class="tg-0pky"><b>Predicate</b></th>
    <th class="tg-0pky"><b>Object</b></th>
  </tr>
</thead>

<tbody>
REPLACE_DATA
</tbody>

</table>
</div>

'''

In [63]:
qual_triplets_vis = {}
for sample_id, triplet_set in triplet_data.items():
    xml_set = ''
    for this_set in triplet_set:
        for item in this_set:
            subj, pred, obj = item
            this_xml_triplet = html_triplet.replace('SUBJ', subj).replace('PRED', pred).replace('OBJ', obj)
            xml_set += this_xml_triplet
    full_xml_table = html_head.replace('REPLACE_DATA', xml_set)
    qual_triplets_vis[sample_id] = full_xml_table

In [66]:
len(qual_triplets_vis)

178

In [65]:
with open('../inputs/triples_REF.json', 'w') as f0:
    json.dump(qual_triplets_vis, f0)